### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-09-02


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-09-02
Yesterday: 2025-09-01


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-09-01'
ORDER BY a.name



(16, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-09-01' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-29,2025-09-01,9.60,9.85,319820,9.85,9.60,B,SET,0.25,2.60
1,CPF,2025-09-01,2025-09-01,21.90,22.40,164889564,24.50,21.90,B,SET50,0.50,2.28
2,GVREIT,2025-08-27,2025-09-01,6.20,6.25,4407728,6.25,6.15,B,SET,0.05,0.81
3,JMART,2025-08-29,2025-09-01,8.25,8.40,19998118,8.50,8.25,B,SET50,0.15,1.82
4,JMT,2025-08-29,2025-09-01,11.70,11.60,19434812,12.00,11.60,B,SET50,-0.10,-0.85
5,ORI,2025-09-01,2025-09-01,2.18,2.20,26957679,2.32,2.16,B,SET100,0.02,0.92
6,PTT,2025-09-01,2025-09-01,31.00,31.50,235403293,32.25,31.00,B,SET50,0.50,1.61
7,RCL,2025-08-28,2025-09-01,28.00,27.50,7657688,28.25,27.50,B,SET100,-0.50,-1.79
8,TFFIF,2025-08-21,2025-09-01,6.05,5.90,29964966,6.10,5.85,B,SET,-0.15,-2.48
9,WHAIR,2025-08-28,2025-09-01,5.55,5.55,2697818,5.60,5.45,B,SET,0.00,0.00


In [20]:
df.shape

(16, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AIMIRT,2025-08-29,2025-09-01,9.60,9.85,"319,820",9.85,9.60,2.60%,B
1,CPF,2025-09-01,2025-09-01,21.90,22.40,"164,889,564",24.50,21.90,2.28%,B
2,GVREIT,2025-08-27,2025-09-01,6.20,6.25,"4,407,728",6.25,6.15,0.81%,B
3,JMART,2025-08-29,2025-09-01,8.25,8.40,"19,998,118",8.50,8.25,1.82%,B
4,JMT,2025-08-29,2025-09-01,11.70,11.60,"19,434,812",12.00,11.60,-0.85%,B
5,ORI,2025-09-01,2025-09-01,2.18,2.20,"26,957,679",2.32,2.16,0.92%,B
6,PTT,2025-09-01,2025-09-01,31.00,31.50,"235,403,293",32.25,31.00,1.61%,B
7,RCL,2025-08-28,2025-09-01,28.00,27.50,"7,657,688",28.25,27.50,-1.79%,B
8,TFFIF,2025-08-21,2025-09-01,6.05,5.90,"29,964,966",6.10,5.85,-2.48%,B
9,WHAIR,2025-08-28,2025-09-01,5.55,5.55,"2,697,818",5.60,5.45,0.00%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(16, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(16, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AIMIRT,2025-08-29,2025-09-01,9.60,9.85,319820,9.85,9.60,2.60,B,9.75,-0.10,131645,2025-09-02
1,CPF,2025-09-01,2025-09-01,21.90,22.40,164889564,24.50,21.90,2.28,B,22.40,0.00,17558977,2025-09-02
2,GVREIT,2025-08-27,2025-09-01,6.20,6.25,4407728,6.25,6.15,0.81,B,6.20,-0.05,410801,2025-09-02
3,JMART,2025-08-29,2025-09-01,8.25,8.40,19998118,8.50,8.25,1.82,B,8.50,0.10,7660842,2025-09-02
4,JMT,2025-08-29,2025-09-01,11.70,11.60,19434812,12.00,11.60,-0.85,B,11.70,0.10,8658231,2025-09-02
5,ORI,2025-09-01,2025-09-01,2.18,2.20,26957679,2.32,2.16,0.92,B,2.22,0.02,4663439,2025-09-02
6,PTT,2025-09-01,2025-09-01,31.00,31.50,235403293,32.25,31.00,1.61,B,31.50,0.00,47215888,2025-09-02
7,RCL,2025-08-28,2025-09-01,28.00,27.50,7657688,28.25,27.50,-1.79,B,27.00,-0.50,3907663,2025-09-02
8,TFFIF,2025-08-21,2025-09-01,6.05,5.90,29964966,6.10,5.85,-2.48,B,5.85,-0.05,2047822,2025-09-02
9,WHAIR,2025-08-28,2025-09-01,5.55,5.55,2697818,5.60,5.45,0.00,B,5.50,-0.05,189921,2025-09-02


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-09-02 2025-09-02   9.85   9.75  -1.02%     319,820   9.85   9.60 B Insert
CPF      2025-09-01 2025-09-02  21.90  22.40   2.28% 164,889,564  24.50  21.90 B Update
DIF      2025-08-26 2025-09-02   8.20   8.25   0.61%  57,994,742   8.30   8.15 S Update
GVREIT   2025-09-02 2025-09-02   6.25   6.20   -0.8%   4,407,728   6.25   6.15 B Insert
IVL      2025-09-01 2025-09-02  21.70  21.90   0.92%  52,159,492  23.40  21.70 S Update
JMART    2025-08-29 2025-09-02   8.25   8.50   3.03%  19,998,118   8.50   8.25 B Update
JMT      2025-09-02 2025-09-02  11.60  11.70   0.86%  19,434,812  12.00  11.60 B Insert
MCS      2025-09-01 2025-09-02   8.35   8.25   -1.2%     604,188   8.40   8.15 S Update
NER      2025-08-29 2025-09-02   4.26   4.30   0.94%   8,870,266   4.32   4.26 S Update
ORI      2025-09-01 2025-09-02  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '{}' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name
""".format(today)
print(sql)


    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '2025-09-02' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(16, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-09-02,2025-09-02,9.85,9.75,"319,820",9.85,9.60,B,SET,-0.10,-1.02%
1,CPF,2025-09-01,2025-09-02,21.90,22.40,"329,779,128",24.50,21.90,B,SET50,0.50,2.28%
2,GVREIT,2025-09-02,2025-09-02,6.25,6.20,"4,407,728",6.25,6.15,B,SET,-0.05,-0.80%
3,JMART,2025-08-29,2025-09-02,8.25,8.50,"39,996,236",8.50,8.25,B,SET50,0.25,3.03%
4,JMT,2025-09-02,2025-09-02,11.60,11.70,"19,434,812",12.00,11.60,B,SET50,0.10,0.86%
5,ORI,2025-09-01,2025-09-02,2.18,2.22,"53,915,358",2.32,2.16,B,SET100,0.04,1.83%
6,PTT,2025-09-01,2025-09-02,31.00,31.50,"470,806,586",32.25,31.00,B,SET50,0.50,1.61%
7,RCL,2025-08-28,2025-09-02,28.00,27.00,"15,315,376",28.25,27.00,B,SET100,-1.00,-3.57%
8,TFFIF,2025-08-21,2025-09-02,6.05,5.85,"59,929,932",6.10,5.85,B,SET,-0.20,-3.31%
9,WHAIR,2025-08-28,2025-09-02,5.55,5.50,"5,395,636",5.60,5.45,B,SET,-0.05,-0.90%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-09-02 20:28:19
